In [28]:
import pandas as pd
import os

# a = pd.read_csv('/home/giuseppedefazio/Documenti/kMetaShot_HQ_paper/SRR606249_mock_mash_pair/megahit_ass2ref10_assignment_evaluation/forbin_comparison_classEval2.csv', index_col=0)
# b = pd.read_csv('/home/giuseppedefazio/Documenti/kMetaShot_HQ_paper/SRR606249_mock_mash_pair/megahit_ass2ref10_assignment_evaluation/kmetashot_exp_classEval2.csv', index_col=0)
# b['bin'] = b['bin'].apply( lambda x: "bin.%s" % x.split('.')[1])
# c = b[['bin', 'ass2ref']].merge(a,on='bin', how='outer')

def score_metrics(tp, fn ,tn, fp):
    sensitivity = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(tn+fp)
    f1_score = (2*sensitivity*precision)/(sensitivity+precision)
    fpr = fp/(fp+tn)
    ba = (sensitivity + specificity)/2
    accuracy = (tp+tn)/sum([tp, fn ,tn, fp])
    return sensitivity, specificity, precision, f1_score, fpr, ba, accuracy

assum = {'taxid':  64320, 'species': 37639, 'genus': 3713}

In [26]:
# ### kMetaShot
# kms = c[c['ass2ref'] >= 0.2]
# tp = kms[(kms['taxid_exp_kmetashot'] == True) & (kms['taxid_kmetashot'] != 0)].shape[0]
# fp = kms[(kms['taxid_exp_kmetashot'] == False) & (kms['taxid_kmetashot'] != 0)].shape[0]
# fn = c.shape[0] - kms.shape[0]
# tn = estrain_tot - (tp+fp+fn)
# 
# res = score_metrics(tp, fn, tn, fp)
# print(*res,tp, fp, fn, tn)


0.391304347826087 0.9993778869931723 0.1836734693877551 0.25 0.0006221130068276902 0.6953411174096297 0.9991604477611941 9 40 14 64257


In [53]:
maind = '/home/giuseppedefazio/Documenti/kMetaShot_HQ_paper/kMetaShot_finishing/cami2/'
results = pd.DataFrame()
c = 0
for env in os.listdir(maind):
    aux = os.path.join(maind, env)
    if os.path.isdir(aux):
        for tec in os.listdir(aux):
            for tool in ['kmetashot','camitax','gtdbtk']:
                sr = os.path.join(maind,env, tec,'kmetashot_exp_classEval2')
                # sc = os.path.join(maind,env, tec,'composition')
                sc = os.path.join(maind,env, tec,'forbin_exp_clas_eval')
                # pb = os.path.join('~/Documenti/kMetaShot_HQ_paper/risultati_cami2/',env, 'pacbio','kmetashot_exp_clas_eval')
                for el in os.listdir(sr):
                    xtab = pd.read_csv(os.path.join(sc, el), index_col=0)
                    tab = pd.read_csv(os.path.join(sr, el), index_col=0)
                    tab['bin'] = tab['bin'].apply(lambda x: "bin.%s" % x.split('.')[1])
                    tab = tab[['bin','ass2ref']].merge(xtab, on='bin', how='outer')
                    bins = tab.shape[0]
                    a2r = 0.2
                    # for a2r in np.arange(0,1,0.05):
                    for rank in ['taxid', 'species', 'genus']:
                        if (tool == 'kmetashot') and (rank != 'genus'):
                            tabc = tab.copy()
                            tabc = tabc[tabc['ass2ref'] >= a2r]
                            print('True', tool, rank)
                        else:
                            tabc = tab.copy()
                            print('False', tool, rank)
                        results.loc[c, 'tool'] = tool
                        results.loc[c, 'env'] = env
                        results.loc[c,'tec'] = tec
                        results.loc[c, 'sample'] = el.split('.')[0]
                        # results.loc[c, 'ass2ref'] = a2r
                        results.loc[c, 'rank'] = rank
                        results.loc[c, 'bins'] = bins
                        try:
                            results.loc[c, 'assigned'] = tabc[(tabc["%s_%s" % (rank, tool)] != 0) | (~tabc["%s_%s" % (rank, tool)].isna())].shape[0]
                            # results.loc[c, 'correctly'] = tabc[((tabc["%s_%s" % (rank, tool)] != 0) | (~tabc["%s_%s" % (rank, tool)].isna())) & (tabc["%s_%s" % (rank, tool)] == True )].shape[0]
                            
                            # la valutazione si sposta sui taxa presenti ed assenti all'interno della mock community
                            # exp = comp[~comp[rank].isna()][rank].unique()
                            # print(rank, exp.__len__())
                            tp = tabc[(tabc['%s_exp_%s' % (rank, tool)] == True) & (tabc["%s_%s" % (rank, tool)] != 0)].shape[0]
                            results.loc[c, 'tp'] = tp
                            
                            fp = tabc[(tabc['%s_exp_%s' % (rank, tool)] == False) & ((tabc["%s_%s" % (rank, tool)] != 0) | (~tabc["%s_%s" % (rank, tool)].isna()))].shape[0]
                            results.loc[c, 'fp'] = fp
                            if (tool == 'kmetashot') and (rank != 'genus'):
                                fn = tab.shape[0] - tabc.shape[0]
                            else:
                                fn = tabc[(tabc["%s_%s" % (rank, tool)].isna()) | (tabc["%s_%s" % (rank, tool)] == 0)].shape[0]
                            results.loc[c, 'fn'] = fn
                            
                            tn = assum[rank] - (fn + fp + tp)
                            results.loc[c, 'tn'] = tn
                            
                            out = score_metrics(tp, fn, tn, fp)
                            results.loc[c, 'sensitivity'] = out[0]
                            results.loc[c, 'specificity'] = out[1]
                            results.loc[c, 'precision'] = out[2]
                            results.loc[c, 'f1_score'] = out[3]
                            results.loc[c, 'fpr'] = out[4]
                            results.loc[c, 'ba'] = out[5]
                            results.loc[c, 'accuracy'] = out[6]
                        except KeyError:
                            pass
                        c += 1
results = results[~results['tp'].isna()]
results

True kmetashot taxid
True kmetashot species
False kmetashot genus
True kmetashot taxid
True kmetashot species
False kmetashot genus
True kmetashot taxid
True kmetashot species
False kmetashot genus
True kmetashot taxid
True kmetashot species
False kmetashot genus
True kmetashot taxid
True kmetashot species
False kmetashot genus
True kmetashot taxid
True kmetashot species
False kmetashot genus
True kmetashot taxid
True kmetashot species
False kmetashot genus
True kmetashot taxid
True kmetashot species
False kmetashot genus
True kmetashot taxid
True kmetashot species
False kmetashot genus
True kmetashot taxid
True kmetashot species
False kmetashot genus
False camitax taxid
False camitax species
False camitax genus
False camitax taxid
False camitax species
False camitax genus
False camitax taxid
False camitax species
False camitax genus
False camitax taxid
False camitax species
False camitax genus
False camitax taxid
False camitax species
False camitax genus
False camitax taxid
False cami

,tool,env,tec,sample,rank,bins,assigned,tp,fp,fn,tn,sensitivity,specificity,precision,f1_score,fpr,ba,accuracy
0,kmetashot,Airways,pacbio,sample_8,taxid,56.0,37.0,29.0,8.0,19.0,64264.0,0.604167,0.999876,0.783784,0.682353,0.000124,0.802021,0.999580
1,kmetashot,Airways,pacbio,sample_8,species,56.0,37.0,37.0,0.0,19.0,37583.0,0.660714,1.000000,1.000000,0.795699,0.000000,0.830357,0.999495
2,kmetashot,Airways,pacbio,sample_8,genus,56.0,56.0,55.0,0.0,1.0,3657.0,0.982143,1.000000,1.000000,0.990991,0.000000,0.991071,0.999731
3,kmetashot,Airways,pacbio,sample_9,taxid,77.0,48.0,30.0,18.0,29.0,64243.0,0.508475,0.999720,0.625000,0.560748,0.000280,0.754097,0.999269
4,kmetashot,Airways,pacbio,sample_9,species,77.0,48.0,45.0,3.0,29.0,37562.0,0.608108,0.999920,0.937500,0.737705,0.000080,0.804014,0.999150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,gtdbtk,GI_tract,short_reads,sample_10,genus,32.0,32.0,19.0,8.0,6.0,3680.0,0.760000,0.997831,0.703704,0.730769,0.002169,0.878915,0.996229
544,gtdbtk,GI_tract,short_reads,sample_1,species,21.0,21.0,14.0,6.0,4.0,37615.0,0.777778,0.999841,0.700000,0.736842,0.000159,0.888809,0.999734
545,gtdbtk,GI_tract,short_reads,sample_1,genus,21.0,21.0,17.0,3.0,1.0,3692.0,0.944444,0.999188,0.850000,0.894737,0.000812,0.971816,0.998923
547,gtdbtk,GI_tract,short_reads,sample_0,species,17.0,17.0,10.0,5.0,4.0,37620.0,0.714286,0.999867,0.666667,0.689655,0.000133,0.857076,0.999761


In [54]:
results.to_csv('/home/giuseppedefazio/Documenti/kMetaShot_HQ_paper/kMetaShot_finishing/cami2/cami2_ass2ref_20.csv')
# xtab

In [18]:
import pandas as pd
results = pd.read_csv('/home/giuseppedefazio/Documenti/kMetaShot_HQ_paper/kMetaShot_finishing/cami2/cami2_ass2ref_20.csv', index_col=0)
c = 0
desc_res = pd.DataFrame()
for tool in results['tool'].unique():
    for env in results['env'].unique():
        for tec in results['tec'].unique():
            for rank in results['rank'].unique():
                if rank == 'taxid' and tool in ('camitax', 'gtdbtk'):
                    pass
                else:
                    sub = results[(results['tool'] == tool) & (results['env'] == env) & 
                    (results['tec'] == tec) & (results['rank'] == rank)]
                    for measure in list(results.columns)[11:]:
                        description = sub[measure].describe()
                        # print(description)
                        desc_res.loc[c, 'tool'] = tool
                        desc_res.loc[c, 'env'] = env
                        desc_res.loc[c, 'tec'] = tec
                        desc_res.loc[c, 'rank'] = rank
                        desc_res.loc[c, '%s_median' % measure] = description['50%']
                        desc_res.loc[c, '%s_IQR' % measure] = description['75%'] - description['25%']
                    c += 1
desc_res = desc_res[~desc_res['sensitivity_median'].isna()]                    
desc_res.to_csv('/home/giuseppedefazio/Documenti/kMetaShot_HQ_paper/kMetaShot_finishing/cami2/cami2_ass2ref_20_descres.csv')                    

In [27]:
desc_res[(desc_res['tec'] == 'short_reads') & (desc_res['env'] == 'Airways') & (desc_res['rank'] == 'species')] 

,tool,env,tec,rank,sensitivity_median,sensitivity_IQR,specificity_median,specificity_IQR,precision_median,precision_IQR,f1_score_median,f1_score_IQR,fpr_median,fpr_IQR,ba_median,ba_IQR,accuracy_median,accuracy_IQR
4,kmetashot,Airways,short_reads,species,0.727941,0.126631,1.000000,0.000020,1.000000,0.057692,0.842365,0.101051,0.000000,0.000020,0.863971,0.063335,0.999867,0.000120
20,camitax,Airways,short_reads,species,0.656863,0.090614,0.999787,0.000160,0.629573,0.062669,0.646110,0.071799,0.000213,0.000160,0.828352,0.045184,0.999588,0.000319
32,gtdbtk,Airways,short_reads,species,0.755952,0.135776,0.999987,0.000066,0.970588,0.097727,0.849624,0.131370,0.000013,0.000066,0.877970,0.067898,0.999854,0.000232


In [21]:
desc_res[(desc_res['tec'] == 'pacbio') & (desc_res['env'] == 'GI_tract') & (desc_res['rank'] == 'genus')]

,tool,env,tec,rank,sensitivity_median,sensitivity_IQR,specificity_median,specificity_IQR,precision_median,precision_IQR,f1_score_median,f1_score_IQR,fpr_median,fpr_IQR,ba_median,ba_IQR,accuracy_median,accuracy_IQR
14,kmetashot,GI_tract,pacbio,genus,0.970779,0.051378,0.999729,0.000475,0.958949,0.066876,0.964894,0.031464,0.000271,0.000475,0.985254,0.025418,0.999461,0.000539
27,camitax,GI_tract,pacbio,genus,0.790584,0.121530,0.998373,0.000815,0.790584,0.127448,0.790584,0.124534,0.001627,0.000815,0.894476,0.061035,0.996768,0.001414
39,gtdbtk,GI_tract,pacbio,genus,0.857143,0.032771,0.998373,0.000951,0.791304,0.178559,0.820856,0.126101,0.001627,0.000951,0.927693,0.016759,0.997576,0.000539
